<a href="https://colab.research.google.com/github/netra-poonia/simulation1/blob/main/Branch_Manager_Bonus_Calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Try 1

In [1]:
import pandas as pd
from datetime import datetime, timedelta
from typing import Dict, List, Tuple, Optional
import numpy as np

class NBFCBonusCalculator:
    """
    NBFC Branch Manager Monthly Bonus Calculator

    This class calculates three types of payouts:
    1. Effort Payout (Payout A) - Based on fresh disbursals and logins
    2. Business Payout (Payout B) - Based on monthly business performance vs targets
    3. Milestone Payout (Payout C) - Quarterly evaluation based on consistency
    """

    def __init__(self):
        # Payout A: Effort payout grids
        self.secured_fresh_disbursals = {
            0:  {0: 0,    1: 1400, 3: 1800, 5: 2200, 7: 2400},
            1:  {0: 1200, 1: 1600, 3: 2000, 5: 2500, 7: 2800},
            7:  {0: 1400, 1: 1800, 3: 2500, 5: 3000, 7: 4000},
            10: {0: 1800, 1: 2200, 3: 3000, 5: 3500, 7: 5000}
        }

        self.unsecured_fresh_disbursals = {
            0:  {0: 0,   1: 250, 4: 350, 8: 450,  10: 550,  14: 700},
            1:  {0: 200, 1: 300, 4: 400, 8: 500,  10: 600,  14: 750},
            15: {0: 250, 1: 350, 4: 450, 8: 550,  10: 650,  14: 850},
            20: {0: 300, 1: 400, 4: 500, 8: 600,  10: 700,  14: 1000},
            24: {0: 500, 1: 600, 4: 700, 8: 1000, 10: 1200, 14: 1500}
        }

        # Payout B: Monthly business grids
        self.secured_business = {
            (0, 60): 0,        # <= 60%
            (60, 80): 450,     # 61% - 80%
            (80, 100): 500,    # 81% - 100%
            (100, 150): 600,   # 101% - 150%
            (150, float('inf')): 750   # > 150%
        }

        self.secured_direct_business = {
            (0, 40): 50,       # <= 40%
            (40, 60): 100,     # 41% - 60%
            (60, 80): 150,     # 61% - 80%
            (80, float('inf')): 200    # > 80%
        }

        self.unsecured_business = {
            (0, 60): 0,     # <= 60%
            (60, 80): 300,     # 61% - 80%
            (80, 100): 400,    # 81% - 100%
            (100, 150): 500,   # 101% - 150%
            (150, float('inf')): 600   # > 150%
        }

        # Docking payout for portfolio quality (DPD 25+)
        self.docking_payout = {
            (0, 2): 1.00,     # 100%
            (2, 3): 0.75,     # 75%
            (3, 5): 0.50,     # 50%
            (5, 100): 0.00    # 0%
        }

        # Milestone payout criteria
        self.milestone_criteria = {
            'salary_hike_low_ctc': {  # CTC < 7L
                'Silver': 0.04,
                'Gold': 0.05,
                'Platinum': 0.08
            },
            'salary_hike_high_ctc': {  # CTC >= 7L
                'Silver': 0.03,
                'Gold': 0.04,
                'Platinum': 0.06
            },
            'achievement_months': {
                5: {'Silver': 3, 'Gold': 4, 'Platinum': 5},
                6: {'Silver': 4, 'Gold': 5, 'Platinum': 6},
                7: {'Silver': 5, 'Gold': 6, 'Platinum': 7}
            }
        }

    def calculate_effort_payout(self, tenure_months: int, secured_disbursals: int,
                               unsecured_disbursals: int, secured_logins: int,
                               unsecured_logins: int) -> Dict:
        """
        Calculate Payout A: Effort payout based on fresh disbursals and logins

        Args:
            tenure_months: BM's tenure in months
            secured_disbursals: Number of secured fresh disbursals
            unsecured_disbursals: Number of unsecured fresh disbursals
            secured_logins: Number of secured logins
            unsecured_logins: Number of unsecured logins
        """

        def get_payout_from_grid(grid: Dict, tenure: int, count: int) -> int:
            # Find the appropriate tenure bracket
            tenure_brackets = sorted(grid.keys())
            applicable_tenure = max([t for t in tenure_brackets if t <= tenure])

            # Find the appropriate count bracket
            count_brackets = sorted(grid[applicable_tenure].keys())
            applicable_count = max([c for c in count_brackets if c <= count], default=0)

            return grid[applicable_tenure].get(applicable_count, 0)

        secured_payout = get_payout_from_grid(self.secured_fresh_disbursals,
                                            tenure_months, secured_disbursals)
        unsecured_payout = get_payout_from_grid(self.unsecured_fresh_disbursals,
                                              tenure_months, unsecured_disbursals)

        return {
            'secured_effort_payout': secured_payout,
            'unsecured_effort_payout': unsecured_payout,
            'total_effort_payout': secured_payout + unsecured_payout,
            'details': {
                'secured_disbursals': secured_disbursals,
                'unsecured_disbursals': unsecured_disbursals,
                'tenure_months': tenure_months
            }
        }

    def calculate_business_payout(self, secured_achievement_pct: float,
                                 secured_direct_achievement_pct: float,
                                 unsecured_achievement_pct: float) -> Dict:
        """
        Calculate Payout B: Business payout based on monthly performance vs targets

        Args:
            secured_achievement_pct: Secured business achievement percentage
            secured_direct_achievement_pct: Secured direct business achievement percentage
            unsecured_achievement_pct: Unsecured business achievement percentage
        """

        def get_business_payout(grid: Dict, achievement_pct: float) -> Optional[int]:
            for (min_pct, max_pct), payout in grid.items():
                if min_pct < achievement_pct <= max_pct:
                    return payout
            return 0

        secured_payout = get_business_payout(self.secured_business, secured_achievement_pct)
        secured_direct_payout = get_business_payout(self.secured_direct_business,
                                                   secured_direct_achievement_pct)
        unsecured_payout = get_business_payout(self.unsecured_business, unsecured_achievement_pct)

        # Handle None values
        secured_payout = secured_payout or 0
        secured_direct_payout = secured_direct_payout or 0
        unsecured_payout = unsecured_payout or 0

        total_business_payout = secured_payout + secured_direct_payout + unsecured_payout

        return {
            'secured_business_payout': secured_payout,
            'secured_direct_business_payout': secured_direct_payout,
            'unsecured_business_payout': unsecured_payout,
            'total_business_payout': total_business_payout,
            'immediate_payout': total_business_payout * 0.85,  # 85% paid immediately
            'deferred_payout': total_business_payout * 0.15,   # 15% accumulated for quality check
            'details': {
                'secured_achievement_pct': secured_achievement_pct,
                'secured_direct_achievement_pct': secured_direct_achievement_pct,
                'unsecured_achievement_pct': unsecured_achievement_pct
            }
        }

    def calculate_docking_release(self, deferred_amount: float, dpd_25_plus_pct: float) -> Dict:
        """
        Calculate the release of deferred business payout based on portfolio quality

        Args:
            deferred_amount: Amount deferred from business payout
            dpd_25_plus_pct: Percentage of portfolio in DPD 25+
        """

        release_factor = 0
        for (min_dpd, max_dpd), factor in self.docking_payout.items():
            if min_dpd < dpd_25_plus_pct <= max_dpd:
                release_factor = factor
                break

        released_amount = deferred_amount * release_factor

        return {
            'deferred_amount': deferred_amount,
            'dpd_25_plus_pct': dpd_25_plus_pct,
            'release_factor': release_factor,
            'released_amount': released_amount,
            'forfeited_amount': deferred_amount - released_amount
        }

    def calculate_milestone_payout(self, doj: datetime, current_date: datetime,
                                  monthly_achievements: List[bool], current_ctc: float) -> Dict:
        """
        Calculate Payout C: Milestone payout based on quarterly evaluation

        Args:
            doj: Date of joining
            current_date: Current evaluation date
            monthly_achievements: List of boolean values indicating 120% target achievement
            current_ctc: Current annual CTC
        """

        # Determine if DOJ month should be counted
        doj_counts = doj.day <= 10

        # Calculate tenure and evaluation period
        if doj_counts:
            start_month = doj.replace(day=1)
        else:
            start_month = (doj.replace(day=1) + timedelta(days=32)).replace(day=1)

        months_diff = (current_date.year - start_month.year) * 12 + (current_date.month - start_month.month)

        # Determine evaluation period length
        if months_diff <= 7:
            eval_period = min(max(months_diff, 5), 7)  # First evaluation: 5-7 months
            criteria = self.milestone_criteria['achievement_months'][eval_period]
        else:
            eval_period = 6  # Ongoing evaluations: 6 months
            criteria = {'Silver': 3, 'Gold': 4, 'Platinum': 4}  # Ongoing criteria

        # Count achievements in the evaluation period
        achievements_count = sum(monthly_achievements[-eval_period:])

        # Determine tier
        tier = None
        if achievements_count >= criteria['Platinum']:
            tier = 'Platinum'
        elif achievements_count >= criteria['Gold']:
            tier = 'Gold'
        elif achievements_count >= criteria['Silver']:
            tier = 'Silver'

        # Calculate salary hike
        salary_hike_pct = 0
        if tier:
            if current_ctc < 700000:  # < 7L
                salary_hike_pct = self.milestone_criteria['salary_hike_low_ctc'][tier]
            else:  # >= 7L
                salary_hike_pct = self.milestone_criteria['salary_hike_high_ctc'][tier]

        return {
            'tier': tier,
            'salary_hike_pct': salary_hike_pct,
            'salary_hike_amount': current_ctc * salary_hike_pct,
            'evaluation_period_months': eval_period,
            'achievements_count': achievements_count,
            'required_achievements': criteria,
            'details': {
                'doj': doj,
                'doj_counts': doj_counts,
                'months_tenure': months_diff,
                'current_ctc': current_ctc
            }
        }

    def calculate_monthly_bonus(self, bm_data: Dict) -> Dict:
        """
        Calculate complete monthly bonus for a Branch Manager

        Args:
            bm_data: Dictionary containing all BM performance data
        """

        # Calculate Effort Payout
        effort_payout = self.calculate_effort_payout(
            bm_data['tenure_months'],
            bm_data['secured_disbursals'],
            bm_data['unsecured_disbursals'],
            bm_data['secured_logins'],
            bm_data['unsecured_logins']
        )

        # Calculate Business Payout
        business_payout = self.calculate_business_payout(
            bm_data['secured_achievement_pct'],
            bm_data['secured_direct_achievement_pct'],
            bm_data['unsecured_achievement_pct']
        )

        # Calculate total monthly payout
        total_monthly_payout = (effort_payout['total_effort_payout'] +
                              business_payout['immediate_payout'])

        return {
            'effort_payout': effort_payout,
            'business_payout': business_payout,
            'total_monthly_payout': total_monthly_payout,
            'deferred_amount': business_payout['deferred_payout'],
            'bm_details': {
                'name': bm_data.get('name', 'Unknown'),
                'employee_id': bm_data.get('employee_id', 'Unknown'),
                'month': bm_data.get('month', 'Unknown')
            }
        }

In [3]:
# Example usage and demonstration
def demonstrate_calculator():
    """
    Demonstrate the bonus calculator with example cases
    """
    calculator = NBFCBonusCalculator()

    print("=" * 80)
    print("NBFC BRANCH MANAGER BONUS CALCULATOR DEMONSTRATION")
    print("=" * 80)

    # Example Case 1: High Performer
    print("\n🎯 EXAMPLE CASE 1: HIGH PERFORMING BRANCH MANAGER")
    print("-" * 50)

    bm_case1 = {
        'name': 'Rajesh Kumar',
        'employee_id': 'BM001',
        'month': 'May 2025',
        # Effort Payout Data
        'tenure_months': 2,
        'secured_disbursals': 5,
        'unsecured_disbursals': 0,
        'secured_logins': 14,
        'unsecured_logins': 1,
        # Business Payout Data
        'secured_achievement_pct': 91,  # 125.5% of target
        'secured_direct_achievement_pct': 100,  # 85.2% of target
        'unsecured_achievement_pct': 0,  # 110.8% of target
    }

    result1 = calculator.calculate_monthly_bonus(bm_case1)

    print(f"Branch Manager: {bm_case1['name']} ({bm_case1['employee_id']})")
    print(f"Month: {bm_case1['month']}")
    print(f"Tenure: {bm_case1['tenure_months']} months")
    print()

    # Effort Payout Details
    print("💰 EFFORT PAYOUT (Payout A):")
    print(f"  • Secured Disbursals: {bm_case1['secured_disbursals']} → ₹{result1['effort_payout']['secured_effort_payout']:,}")
    print(f"  • Unsecured Disbursals: {bm_case1['unsecured_disbursals']} → ₹{result1['effort_payout']['unsecured_effort_payout']:,}")
    print(f"  • Total Effort Payout: ₹{result1['effort_payout']['total_effort_payout']:,}")
    print()

    # Business Payout Details
    print("📈 BUSINESS PAYOUT (Payout B):")
    print(f"  • Secured Business ({bm_case1['secured_achievement_pct']:.1f}%): ₹{result1['business_payout']['secured_business_payout']:,}")
    print(f"  • Secured Direct ({bm_case1['secured_direct_achievement_pct']:.1f}%): ₹{result1['business_payout']['secured_direct_business_payout']:,}")
    print(f"  • Unsecured Business ({bm_case1['unsecured_achievement_pct']:.1f}%): ₹{result1['business_payout']['unsecured_business_payout']:,}")
    print(f"  • Total Business Payout: ₹{result1['business_payout']['total_business_payout']:,}")
    print(f"  • Immediate Payout (85%): ₹{result1['business_payout']['immediate_payout']:,.0f}")
    print(f"  • Deferred for Quality Check (15%): ₹{result1['business_payout']['deferred_payout']:,.0f}")
    print()

    print("💵 MONTHLY BONUS SUMMARY:")
    print(f"  • Total Monthly Payout: ₹{result1['total_monthly_payout']:,.0f}")
    print(f"  • Amount Deferred: ₹{result1['deferred_amount']:,.0f}")
    print()

    # Example Case 2: Average Performer
    print("\n📊 EXAMPLE CASE 2: AVERAGE PERFORMING BRANCH MANAGER")
    print("-" * 50)

    bm_case2 = {
        'name': 'Priya Sharma',
        'employee_id': 'BM002',
        'month': 'January 2025',
        'tenure_months': 3,
        'secured_disbursals': 1,
        'unsecured_disbursals': 0,
        'secured_logins': 20,
        'unsecured_logins': 0,
        'secured_achievement_pct': 60,
        'secured_direct_achievement_pct': 100,
        'unsecured_achievement_pct': 0,
    }

    result2 = calculator.calculate_monthly_bonus(bm_case2)

    print(f"Branch Manager: {bm_case2['name']} ({bm_case2['employee_id']})")
    print(f"Month: {bm_case2['month']}")
    print(f"Tenure: {bm_case2['tenure_months']} months")
    print()

    print("💰 EFFORT PAYOUT (Payout A):")
    print(f"  • Total Effort Payout: ₹{result2['effort_payout']['total_effort_payout']:,}")
    print()

    print("📈 BUSINESS PAYOUT (Payout B):")
    print(f"  • Total Business Payout: ₹{result2['business_payout']['total_business_payout']:,}")
    print(f"  • Immediate Payout (85%): ₹{result2['business_payout']['immediate_payout']:,.0f}")
    print()

    print("💵 MONTHLY BONUS SUMMARY:")
    print(f"  • Total Monthly Payout: ₹{result2['total_monthly_payout']:,.0f}")
    print()

    # Docking Example
    print("\n🔍 EXAMPLE: DEFERRED PAYOUT RELEASE (Quality Check)")
    print("-" * 50)

    # Simulate Q+2 evaluation for Case 1's deferred amount
    dpd_scenarios = [1.5, 2.8, 4.2, 6.1]
    deferred_amount = result1['deferred_amount']

    print(f"Deferred Amount from Previous Quarter: ₹{deferred_amount:,.0f}")
    print()

    for dpd_pct in dpd_scenarios:
        docking_result = calculator.calculate_docking_release(deferred_amount, dpd_pct)
        print(f"  • DPD 25+ at {dpd_pct:.1f}%:")
        print(f"    - Release Factor: {docking_result['release_factor']:.0%}")
        print(f"    - Amount Released: ₹{docking_result['released_amount']:,.0f}")
        print(f"    - Amount Forfeited: ₹{docking_result['forfeited_amount']:,.0f}")
        print()

    # Milestone Payout Example
    print("\n🏆 EXAMPLE: MILESTONE PAYOUT (Payout C)")
    print("-" * 50)

    from datetime import datetime

    milestone_data = {
        'doj': datetime(2024, 8, 5),  # Joined Aug 5, 2024
        'current_date': datetime(2025, 1, 1),  # Evaluation in Jan 2025
        'monthly_achievements': [True, True, False, True, True, True],  # 6 months, 5 achievements
        'current_ctc': 650000  # 6.5L CTC
    }

    milestone_result = calculator.calculate_milestone_payout(
        milestone_data['doj'],
        milestone_data['current_date'],
        milestone_data['monthly_achievements'],
        milestone_data['current_ctc']
    )

    print(f"Branch Manager DOJ: {milestone_data['doj'].strftime('%B %d, %Y')}")
    print(f"Evaluation Date: {milestone_data['current_date'].strftime('%B %d, %Y')}")
    print(f"Current CTC: ₹{milestone_data['current_ctc']:,}")
    print(f"Evaluation Period: {milestone_result['evaluation_period_months']} months")
    print(f"Target Achievements: {milestone_result['achievements_count']}/{milestone_result['evaluation_period_months']}")
    print()

    if milestone_result['tier']:
        print(f"🎉 MILESTONE ACHIEVED: {milestone_result['tier'].upper()}")
        print(f"  • Salary Hike: {milestone_result['salary_hike_pct']:.1%}")
        print(f"  • Hike Amount: ₹{milestone_result['salary_hike_amount']:,.0f}")
    else:
        print("❌ No milestone achieved this quarter")

    print("\n" + "=" * 80)
    print("CALCULATION COMPLETE")
    print("=" * 80)

# Run the demonstration
if __name__ == "__main__":
    demonstrate_calculator()

NBFC BRANCH MANAGER BONUS CALCULATOR DEMONSTRATION

🎯 EXAMPLE CASE 1: HIGH PERFORMING BRANCH MANAGER
--------------------------------------------------
Branch Manager: Rajesh Kumar (BM001)
Month: May 2025
Tenure: 2 months

💰 EFFORT PAYOUT (Payout A):
  • Secured Disbursals: 5 → ₹2,500
  • Unsecured Disbursals: 0 → ₹200
  • Total Effort Payout: ₹2,700

📈 BUSINESS PAYOUT (Payout B):
  • Secured Business (91.0%): ₹500
  • Secured Direct (100.0%): ₹200
  • Unsecured Business (0.0%): ₹0
  • Total Business Payout: ₹700
  • Immediate Payout (85%): ₹595
  • Deferred for Quality Check (15%): ₹105

💵 MONTHLY BONUS SUMMARY:
  • Total Monthly Payout: ₹3,295
  • Amount Deferred: ₹105


📊 EXAMPLE CASE 2: AVERAGE PERFORMING BRANCH MANAGER
--------------------------------------------------
Branch Manager: Priya Sharma (BM002)
Month: January 2025
Tenure: 3 months

💰 EFFORT PAYOUT (Payout A):
  • Total Effort Payout: ₹1,800

📈 BUSINESS PAYOUT (Payout B):
  • Total Business Payout: ₹200
  • Immediate Pay

## Try 2

In [8]:
from datetime import datetime, timedelta
from typing import Dict, List, Optional, Tuple

class BMBonusCalculator:
    """
    Branch Manager Monthly Bonus Calculator - Range-Based Grid System

    Three types of payouts:
    1. Effort Payout - Based on tenure ranges and disbursal/login ranges (NOT sum)
    2. Business Payout - Tax slab style calculation
    3. Milestone Payout - Quarterly evaluation
    """

    def __init__(self):
        # Effort payout grids with range-based lookup (tenure_range: {disbursal_range: payout})
        self.secured_fresh_disbursals = {
            (0, 0):  {(0, 0): 0,    (1, 2): 1400, (3, 4): 1800, (5, 6): 2200, (7, float('inf')): 2400},
            (1, 6):  {(0, 0): 1200, (1, 2): 1600, (3, 4): 2000, (5, 6): 2500, (7, float('inf')): 2800},
            (7, 9):  {(0, 0): 1400, (1, 2): 1800, (3, 4): 2500, (5, 6): 3000, (7, float('inf')): 4000},
            (10, float('inf')): {(0, 0): 1800, (1, 2): 2200, (3, 4): 3000, (5, 6): 3500, (7, float('inf')): 5000}
        }

        self.unsecured_fresh_disbursals = {
            (0, 0):  {(0, 0): 0,   (1, 3): 250, (4, 7): 350, (8, 9): 450,  (10, 13): 550,  (14, float('inf')): 700},
            (1, 14):  {(0, 0): 200, (1, 3): 300, (4, 7): 400, (8, 9): 500,  (10, 13): 600,  (14, float('inf')): 750},
            (15, 19): {(0, 0): 250, (1, 3): 350, (4, 7): 450, (8, 9): 550,  (10, 13): 650,  (14, float('inf')): 850},
            (20, 23): {(0, 0): 300, (1, 3): 400, (4, 7): 500, (8, 9): 600,  (10, 13): 700,  (14, float('inf')): 1000},
            (24, float('inf')): {(0, 0): 500, (1, 3): 600, (4, 7): 700, (8, 9): 1000, (10, 13): 1200, (14, float('inf')): 1500}
        }

        # Business payout slabs (like income tax slabs)
        self.secured_business_slabs = [
            (0, 60, 0),      # 0-60%: ₹0 per lakh
            (60, 80, 450),   # 60-80%: ₹450 per lakh
            (80, 100, 500),  # 80-100%: ₹500 per lakh
            (100, 150, 600), # 100-150%: ₹600 per lakh
            (150, 1000, 750) # 150%+: ₹750 per lakh
        ]

        self.secured_direct_business_slabs = [
            (0, 40, 50),     # 0-40%: ₹50 fixed
            (40, 60, 100),   # 40-60%: ₹100 fixed
            (60, 80, 150),   # 60-80%: ₹150 fixed
            (80, 1000, 200)  # 80%+: ₹200 fixed
        ]

        self.unsecured_business_slabs = [
            (0, 60, 0),      # 0-60%: ₹0 per lakh
            (60, 80, 300),   # 60-80%: ₹300 per lakh
            (80, 100, 400),  # 80-100%: ₹400 per lakh
            (100, 150, 500), # 100-150%: ₹500 per lakh
            (150, 1000, 600) # 150%+: ₹600 per lakh
        ]

    def find_range_match(self, value: int, range_dict: Dict[Tuple[int, float], any]) -> any:
        """
        Find which range the value falls into and return corresponding value

        Args:
            value: The value to match (tenure, disbursals, or logins)
            range_dict: Dictionary with (min, max) tuple keys

        Returns:
            Matched value or nested dictionary
        """
        for (min_val, max_val), result in range_dict.items():
            if min_val <= value <= max_val:
                return result

        # If no exact match found, return 0 or empty dict
        return 0 if isinstance(list(range_dict.values())[0], (int, float)) else {}

    def calculate_effort_payout(self, tenure_months: int, secured_disbursals: int,
                               unsecured_disbursals: int, secured_logins: int,
                               unsecured_logins: int) -> Dict:
        """
        Calculate effort payout using range-based grid lookup

        Logic:
        1. Find tenure range that matches actual tenure
        2. For that tenure range, find disbursal range that matches actual disbursals
        3. For that tenure range, find login range that matches actual logins
        4. Get separate payouts for disbursals and logins, then add them
        """

        # Find secured payout
        secured_tenure_grid = self.find_range_match(tenure_months, self.secured_fresh_disbursals)
        secured_disbursal_payout = self.find_range_match(secured_disbursals, secured_tenure_grid) if secured_tenure_grid else 0
        secured_login_payout = self.find_range_match(secured_logins, secured_tenure_grid) if secured_tenure_grid else 0
        secured_total_payout = secured_disbursal_payout + secured_login_payout

        # Find unsecured payout
        unsecured_tenure_grid = self.find_range_match(tenure_months, self.unsecured_fresh_disbursals)
        unsecured_disbursal_payout = self.find_range_match(unsecured_disbursals, unsecured_tenure_grid) if unsecured_tenure_grid else 0
        unsecured_login_payout = self.find_range_match(unsecured_logins, unsecured_tenure_grid) if unsecured_tenure_grid else 0
        unsecured_total_payout = unsecured_disbursal_payout + unsecured_login_payout

        return {
            'secured_disbursal_payout': secured_disbursal_payout,
            'secured_login_payout': secured_login_payout,
            'secured_total_payout': secured_total_payout,
            'unsecured_disbursal_payout': unsecured_disbursal_payout,
            'unsecured_login_payout': unsecured_login_payout,
            'unsecured_total_payout': unsecured_total_payout,
            'total_effort_payout': secured_total_payout + unsecured_total_payout,
            'details': {
                'tenure_months': tenure_months,
                'secured_disbursals': secured_disbursals,
                'secured_logins': secured_logins,
                'unsecured_disbursals': unsecured_disbursals,
                'unsecured_logins': unsecured_logins
            }
        }

    def calculate_business_payout_slab_wise(self, achievement_pct: float, business_lakhs: float,
                                          slabs: List, is_fixed_payout: bool = False) -> Dict:
        """
        Calculate business payout using tax slab logic
        """
        total_payout = 0
        breakdown = []

        for min_pct, max_pct, rate in slabs:
            if achievement_pct <= min_pct:
                break

            # Calculate the portion that falls in this slab
            slab_start = max(min_pct, 0)
            slab_end = min(max_pct, achievement_pct)

            if slab_end > slab_start:
                if is_fixed_payout:
                    # Fixed payout regardless of business amount
                    slab_payout = rate
                    breakdown.append({
                        'slab': f"{slab_start}-{slab_end}%",
                        'rate': f"₹{rate} fixed",
                        'payout': slab_payout
                    })
                else:
                    # Per lakh payout - only for the portion in this slab
                    slab_portion = (slab_end - slab_start) / 100
                    slab_business = business_lakhs * slab_portion
                    slab_payout = rate * slab_business
                    breakdown.append({
                        'slab': f"{slab_start}-{slab_end}%",
                        'rate': f"₹{rate}/lakh",
                        'business_lakhs': slab_business,
                        'payout': slab_payout
                    })

                total_payout += slab_payout

        return {
            'total_payout': total_payout,
            'breakdown': breakdown,
            'achievement_pct': achievement_pct
        }

    def calculate_business_payout(self, secured_business_amount: float, secured_target: float,
                                 secured_direct_achievement_pct: float,
                                 unsecured_business_amount: float, unsecured_target: float) -> Dict:
        """Calculate complete business payout"""

        # Calculate achievement percentages
        secured_achievement_pct = (secured_business_amount / secured_target) * 100
        unsecured_achievement_pct = (unsecured_business_amount / unsecured_target) * 100

        # Convert to lakhs
        secured_lakhs = secured_business_amount / 100000
        unsecured_lakhs = unsecured_business_amount / 100000

        # Calculate slab-wise payouts
        secured_result = self.calculate_business_payout_slab_wise(
            secured_achievement_pct, secured_lakhs, self.secured_business_slabs)

        secured_direct_result = self.calculate_business_payout_slab_wise(
            secured_direct_achievement_pct, 0, self.secured_direct_business_slabs, is_fixed_payout=True)

        unsecured_result = self.calculate_business_payout_slab_wise(
            unsecured_achievement_pct, unsecured_lakhs, self.unsecured_business_slabs)

        total_business_payout = (secured_result['total_payout'] +
                               secured_direct_result['total_payout'] +
                               unsecured_result['total_payout'])

        return {
            'secured_payout': secured_result['total_payout'],
            'secured_direct_payout': secured_direct_result['total_payout'],
            'unsecured_payout': unsecured_result['total_payout'],
            'total_business_payout': total_business_payout,
            'immediate_payout': total_business_payout * 0.85,
            'deferred_payout': total_business_payout * 0.15,
            'details': {
                'secured_breakdown': secured_result['breakdown'],
                'secured_direct_breakdown': secured_direct_result['breakdown'],
                'unsecured_breakdown': unsecured_result['breakdown'],
                'secured_achievement_pct': secured_achievement_pct,
                'unsecured_achievement_pct': unsecured_achievement_pct
            }
        }

    def calculate_monthly_bonus(self, bm_data: Dict) -> Dict:
        """Calculate complete monthly bonus"""

        effort_payout = self.calculate_effort_payout(
            bm_data['tenure_months'],
            bm_data['secured_disbursals'],
            bm_data['unsecured_disbursals'],
            bm_data['secured_logins'],
            bm_data['unsecured_logins']
        )

        business_payout = self.calculate_business_payout(
            bm_data['secured_business_amount'],
            bm_data['secured_target'],
            bm_data['secured_direct_achievement_pct'],
            bm_data['unsecured_business_amount'],
            bm_data['unsecured_target']
        )

        total_monthly_payout = effort_payout['total_effort_payout'] + business_payout['immediate_payout']

        return {
            'effort_payout': effort_payout,
            'business_payout': business_payout,
            'total_monthly_payout': total_monthly_payout,
            'deferred_amount': business_payout['deferred_payout']
        }

# Demonstration with your specific example
def demonstrate_calculator():
    calculator = BMBonusCalculator()

    print("=" * 80)
    print("RANGE-BASED BRANCH MANAGER BONUS CALCULATOR")
    print("=" * 80)

    # Your specific case
    bm_data = {
        'tenure_months': 10,      # Falls in (10, inf) range
        'secured_disbursals': 5,  # Falls in (5, 6) range
        'unsecured_disbursals': 0, # Falls in (0, 0) range
        'secured_logins': 14,     # Falls in (7, inf) range
        'unsecured_logins': 1,    # Falls in (1, 3) range
        'secured_business_amount': 3906813,
        'secured_target': 4275000,
        'secured_direct_achievement_pct': 100.0,
        'unsecured_business_amount': 0,
        'unsecured_target': 225000,
    }

    result = calculator.calculate_monthly_bonus(bm_data)

    print(f"\n🎯 YOUR EXAMPLE CASE")
    print(f"Tenure: {bm_data['tenure_months']} months (Range: 10-∞)")
    print()

    # Effort Payout Breakdown
    effort = result['effort_payout']
    print("💰 EFFORT PAYOUT (Range-Based Lookup):")
    print(f"  SECURED:")
    print(f"    • Disbursals: {bm_data['secured_disbursals']} → Range (5,6) → ₹{effort['secured_disbursal_payout']:,}")
    print(f"    • Logins: {bm_data['secured_logins']} → Range (7,∞) → ₹{effort['secured_login_payout']:,}")
    print(f"    • Secured Total: ₹{effort['secured_total_payout']:,}")
    print()
    print(f"  UNSECURED:")
    print(f"    • Disbursals: {bm_data['unsecured_disbursals']} → Range (0,0) → ₹{effort['unsecured_disbursal_payout']:,}")
    print(f"    • Logins: {bm_data['unsecured_logins']} → Range (1,3) → ₹{effort['unsecured_login_payout']:,}")
    print(f"    • Unsecured Total: ₹{effort['unsecured_total_payout']:,}")
    print()
    print(f"  TOTAL EFFORT PAYOUT: ₹{effort['total_effort_payout']:,}")
    print()

    # Business Payout
    business = result['business_payout']
    print("📈 BUSINESS PAYOUT (Tax Slab Style):")
    print(f"  Secured Achievement: {business['details']['secured_achievement_pct']:.1f}%")
    print(f"  Business Amount: ₹{bm_data['secured_business_amount']:,} = {bm_data['secured_business_amount']/100000:.2f} lakhs")
    print("  Slab-wise calculation:")

    for slab in business['details']['secured_breakdown']:
        if 'business_lakhs' in slab:
            print(f"    {slab['slab']}: {slab['business_lakhs']:.2f} lakhs × {slab['rate']} = ₹{slab['payout']:,.0f}")
        else:
            print(f"    {slab['slab']}: {slab['rate']} = ₹{slab['payout']:,.0f}")

    print(f"  Secured Total: ₹{business['secured_payout']:,.0f}")
    print(f"  Direct Payout: ₹{business['secured_direct_payout']:,.0f}")
    print(f"  Total Business: ₹{business['total_business_payout']:,.0f}")
    print()

    print("💵 FINAL MONTHLY BONUS:")
    print(f"  Effort Payout: ₹{effort['total_effort_payout']:,}")
    print(f"  Business Payout (85%): ₹{business['immediate_payout']:,.0f}")
    print(f"  TOTAL MONTHLY: ₹{result['total_monthly_payout']:,.0f}")
    print(f"  Deferred (15%): ₹{business['deferred_payout']:,.0f}")
    print()

    # Verify expected values
    print("🔍 VERIFICATION:")
    print(f"  Expected Secured Effort: ₹3,500 + ₹5,000 = ₹8,500")
    print(f"  Expected Unsecured Effort: ₹200 + ₹300 = ₹500")
    print(f"  Calculated Total Effort: ₹{effort['total_effort_payout']:,}")

    # Range explanation
    print(f"\n📋 RANGE EXPLANATION:")
    print(f"  Tenure {bm_data['tenure_months']} months → (10,∞) range")
    print(f"  Secured disbursals {bm_data['secured_disbursals']} → (5,6) range → ₹3,500")
    print(f"  Secured logins {bm_data['secured_logins']} → (7,∞) range → ₹5,000")
    print(f"  Unsecured disbursals {bm_data['unsecured_disbursals']} → (0,0) range → ₹200")
    print(f"  Unsecured logins {bm_data['unsecured_logins']} → (1,3) range → ₹300")

def explain_range_logic():
    print(f"\n🔍 HOW RANGE-BASED GRID SYSTEM WORKS")
    print("=" * 80)

    print("CORRECT LOGIC:")
    print("1. Find tenure range: 10 months → (10,∞) range")
    print("2. Within that tenure range, find separate payouts for:")
    print("   - Disbursals: 5 → (5,6) range → ₹3,500")
    print("   - Logins: 14 → (7,∞) range → ₹5,000")
    print("3. Add both payouts: ₹3,500 + ₹5,000 = ₹8,500")
    print()
    print("WRONG LOGIC (Previous):")
    print("1. Add disbursals + logins: 5 + 14 = 19")
    print("2. Find payout for total count 19 → Wrong approach!")
    print()
    print("KEY INSIGHT:")
    print("• Disbursals and logins have SEPARATE ranges and payouts")
    print("• They are NOT summed together")
    print("• Each gets evaluated independently within the tenure range")

if __name__ == "__main__":
    demonstrate_calculator()
    explain_range_logic()

RANGE-BASED BRANCH MANAGER BONUS CALCULATOR

🎯 YOUR EXAMPLE CASE
Tenure: 10 months (Range: 10-∞)

💰 EFFORT PAYOUT (Range-Based Lookup):
  SECURED:
    • Disbursals: 5 → Range (5,6) → ₹3,500
    • Logins: 14 → Range (7,∞) → ₹5,000
    • Secured Total: ₹8,500

  UNSECURED:
    • Disbursals: 0 → Range (0,0) → ₹200
    • Logins: 1 → Range (1,3) → ₹300
    • Unsecured Total: ₹500

  TOTAL EFFORT PAYOUT: ₹9,000

📈 BUSINESS PAYOUT (Tax Slab Style):
  Secured Achievement: 91.4%
  Business Amount: ₹3,906,813 = 39.07 lakhs
  Slab-wise calculation:
    0-60%: 23.44 lakhs × ₹0/lakh = ₹0
    60-80%: 7.81 lakhs × ₹450/lakh = ₹3,516
    80-91.38743859649124%: 4.45 lakhs × ₹500/lakh = ₹2,224
  Secured Total: ₹5,741
  Direct Payout: ₹500
  Total Business: ₹6,241

💵 FINAL MONTHLY BONUS:
  Effort Payout: ₹9,000
  Business Payout (85%): ₹5,304
  TOTAL MONTHLY: ₹14,304
  Deferred (15%): ₹936

🔍 VERIFICATION:
  Expected Secured Effort: ₹3,500 + ₹5,000 = ₹8,500
  Expected Unsecured Effort: ₹200 + ₹300 = ₹500

## Try 3

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from typing import Dict, List, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

In [10]:
class BMIncentiveCalculator:

    def __init__(self):
        self.secured_fresh_disbursals = {
            (0, 0):  {(0, 0): 0,    (1, 2): 1400, (3, 4): 1800, (5, 6): 2200, (7, float('inf')): 2400},
            (1, 6):  {(0, 0): 1200, (1, 2): 1600, (3, 4): 2000, (5, 6): 2500, (7, float('inf')): 2800},
            (7, 9):  {(0, 0): 1400, (1, 2): 1800, (3, 4): 2500, (5, 6): 3000, (7, float('inf')): 4000},
            (10, float('inf')): {(0, 0): 1800, (1, 2): 2200, (3, 4): 3000, (5, 6): 3500, (7, float('inf')): 5000}
        }

        self.unsecured_fresh_disbursals = {
            (0, 0):  {(0, 0): 0,   (1, 3): 250, (4, 7): 350, (8, 9): 450,  (10, 13): 550,  (14, float('inf')): 700},
            (1, 14):  {(0, 0): 200, (1, 3): 300, (4, 7): 400, (8, 9): 500,  (10, 13): 600,  (14, float('inf')): 750},
            (15, 19): {(0, 0): 250, (1, 3): 350, (4, 7): 450, (8, 9): 550,  (10, 13): 650,  (14, float('inf')): 850},
            (20, 23): {(0, 0): 300, (1, 3): 400, (4, 7): 500, (8, 9): 600,  (10, 13): 700,  (14, float('inf')): 1000},
            (24, float('inf')): {(0, 0): 500, (1, 3): 600, (4, 7): 700, (8, 9): 1000, (10, 13): 1200, (14, float('inf')): 1500}
        }

        self.secured_business_slabs = [
            (0, 60, 0),
            (60, 80, 450),
            (80, 100, 500),
            (100, 150, 600),
            (150, 1000, 750)
        ]

        self.secured_direct_business_slabs = [
            (0, 40, 50),
            (40, 60, 100),
            (60, 80, 150),
            (80, 1000, 200)
        ]

        self.unsecured_business_slabs = [
            (0, 60, 0),
            (60, 80, 300),
            (80, 100, 400),
            (100, 150, 500),
            (150, 1000, 600)
        ]

        self.docking_payout = {
            (0, 2): 1.00,
            (2, 3): 0.75,
            (3, 5): 0.50,
            (5, 100): 0.00
        }

        self.milestone_hikes = {
            'low_ctc': {  # CTC < 7L
                'Silver': 0.04,
                'Gold': 0.05,
                'Platinum': 0.08
            },
            'high_ctc': {  # CTC >= 7L
                'Silver': 0.03,
                'Gold': 0.04,
                'Platinum': 0.06
            }
        }

    def find_tenure_slab(self, tenure_months: int, grid: Dict) -> Optional[Dict]:
        """Find the appropriate tenure slab in the grid"""
        for (start, end), disbursals in grid.items():
            if start <= tenure_months <= end:
                return disbursals
        return None

    def find_disbursal_slab(self, disbursals: int, disbursal_grid: Dict) -> int:
        """Find the appropriate disbursal slab and return payout"""
        for (start, end), payout in disbursal_grid.items():
            if start <= disbursals <= end:
                return payout
        return 0

    def find_business_slab(self, achievement_pct: float, range_tuple: Tuple) -> bool:
        """Check if achievement percentage falls in the given range"""
        start, end = range_tuple
        return start < achievement_pct <= end

    def calculate_slab_based_payout(self, achievement_pct: float, business_amount_lakhs: float, slabs: List) -> float:
        """Calculate payout based on achievement slabs (like income tax calculation)"""
        total_payout = 0

        for start, end, rate in slabs:
            if achievement_pct <= start:
                break

            # Calculate the portion of business in this slab
            slab_achievement = min(achievement_pct, end) - start

            if slab_achievement > 0:
                if start == 0 and end <= 100:  # Percentage-based slabs
                    slab_business = (slab_achievement / 100) * business_amount_lakhs
                else:  # Fixed amount slabs
                    slab_business = business_amount_lakhs

                total_payout += slab_business * rate

        return total_payout

    def calculate_effort_payout(self, tenure_months: int, secured_disbursals: int, unsecured_disbursals: int) -> Dict:
        """Calculate effort-based payout for fresh disbursals"""

        # Calculate secured effort payout
        secured_grid = self.find_tenure_slab(tenure_months, self.secured_fresh_disbursals)
        secured_payout = 0
        if secured_grid:
            secured_payout = self.find_disbursal_slab(secured_disbursals, secured_grid)

        # Calculate unsecured effort payout
        unsecured_grid = self.find_tenure_slab(tenure_months, self.unsecured_fresh_disbursals)
        unsecured_payout = 0
        if unsecured_grid:
            unsecured_payout = self.find_disbursal_slab(unsecured_disbursals, unsecured_grid)

        return {
            'secured_effort_payout': secured_payout,
            'unsecured_effort_payout': unsecured_payout,
            'total_effort_payout': secured_payout + unsecured_payout
        }

    def calculate_volume_payout(self, secured_achievement_pct: float, secured_business_lakhs: float,
                               secured_direct_achievement_pct: float, secured_direct_business_lakhs: float,
                               unsecured_achievement_pct: float, unsecured_business_lakhs: float) -> Dict:
        """Calculate volume-based business payout"""

        # Calculate secured business payout (slab-based)
        secured_payout = self.calculate_slab_based_payout(
            secured_achievement_pct, secured_business_lakhs, self.secured_business_slabs
        )

        # Calculate secured direct business payout (fixed amounts)
        secured_direct_payout = 0
        for start, end, amount in self.secured_direct_business_slabs:
            if start < secured_direct_achievement_pct <= end:
                secured_direct_payout = amount
                break

        # Calculate unsecured business payout (slab-based)
        unsecured_payout = self.calculate_slab_based_payout(
            unsecured_achievement_pct, unsecured_business_lakhs, self.unsecured_business_slabs
        )

        total_volume_payout = secured_payout + secured_direct_payout + unsecured_payout

        return {
            'secured_business_payout': secured_payout,
            'secured_direct_payout': secured_direct_payout,
            'unsecured_business_payout': unsecured_payout,
            'total_volume_payout': total_volume_payout,
            'immediate_release': total_volume_payout * 0.85,  # 85% released immediately
            'quarterly_accumulated': total_volume_payout * 0.15  # 15% accumulated for quarterly release
        }

    def calculate_docking_release(self, accumulated_amount: float, dpd_25_plus_pct: float) -> Dict:
        """Calculate quarterly docking release based on portfolio quality"""

        release_factor = 0
        for (start, end), factor in self.docking_payout.items():
            if start <= dpd_25_plus_pct < end:
                release_factor = factor
                break

        released_amount = accumulated_amount * release_factor

        return {
            'accumulated_amount': accumulated_amount,
            'dpd_25_plus_percentage': dpd_25_plus_pct,
            'release_factor': release_factor,
            'released_amount': released_amount,
            'forfeited_amount': accumulated_amount - released_amount
        }

    def calculate_monthly_incentive(self, bm_data: Dict) -> Dict:
        """Calculate complete monthly incentive for a Branch Manager"""

        # Calculate effort payout
        effort_result = self.calculate_effort_payout(
            bm_data['tenure_months'],
            bm_data['secured_fresh_disbursals'],
            bm_data['unsecured_fresh_disbursals']
        )

        # Calculate volume payout
        volume_result = self.calculate_volume_payout(
            bm_data['secured_achievement_pct'],
            bm_data['secured_business_lakhs'],
            bm_data['secured_direct_achievement_pct'],
            bm_data['secured_direct_business_lakhs'],
            bm_data['unsecured_achievement_pct'],
            bm_data['unsecured_business_lakhs']
        )

        # Calculate total monthly payout
        total_monthly_payout = effort_result['total_effort_payout'] + volume_result['immediate_release']

        return {
            'bm_name': bm_data.get('bm_name', 'Unknown'),
            'month': bm_data.get('month', 'Unknown'),
            'effort_payout': effort_result,
            'volume_payout': volume_result,
            'total_monthly_payout': total_monthly_payout,
            'quarterly_accumulated': volume_result['quarterly_accumulated']
        }

    def print_detailed_breakdown(self, result: Dict):
        """Print detailed breakdown of incentive calculation"""

        print("="*80)
        print(f"NBFC BRANCH MANAGER INCENTIVE CALCULATION")
        print("="*80)
        print(f"Branch Manager: {result['bm_name']}")
        print(f"Month: {result['month']}")
        print("-"*80)

        # Effort Payout Details
        effort = result['effort_payout']
        print("EFFORT PAYOUT (Fresh Disbursals)")
        print(f"  Secured Effort Payout:    ₹{effort['secured_effort_payout']:,}")
        print(f"  Unsecured Effort Payout:  ₹{effort['unsecured_effort_payout']:,}")
        print(f"  Total Effort Payout:     ₹{effort['total_effort_payout']:,}")
        print()

        # Volume Payout Details
        volume = result['volume_payout']
        print("VOLUME PAYOUT (Business Achievement)")
        print(f"  Secured Business Payout:  ₹{volume['secured_business_payout']:,.2f}")
        print(f"  Secured Direct Payout:    ₹{volume['secured_direct_payout']:,}")
        print(f"  Unsecured Business Payout:₹{volume['unsecured_business_payout']:,.2f}")
        print(f"  Total Volume Payout:     ₹{volume['total_volume_payout']:,.2f}")
        print()

        # Payout Release Details
        print("PAYOUT RELEASE STRUCTURE")
        print(f"  Immediate Release (85%):  ₹{volume['immediate_release']:,.2f}")
        print(f"  Quarterly Accumulation:   ₹{volume['quarterly_accumulated']:,.2f}")
        print()

        # Final Summary
        print("MONTHLY PAYOUT SUMMARY")
        print(f"  Total Monthly Payout:     ₹{result['total_monthly_payout']:,.2f}")
        print("="*80)

In [ ]:
# Example usage and testing
def create_sample_data():
    """Create sample data for testing the calculator"""

    sample_bm = {
        'bm_name': 'Rajesh Kumar',
        'month': 'January 2025',
        'tenure_months': 14,  # 14 months tenure

        # Fresh disbursals for effort payout
        'secured_fresh_disbursals': 5,    # 5 secured disbursals
        'unsecured_fresh_disbursals': 1,   # 1 unsecured disbursal

        # Business achievement for volume payout
        'secured_achievement_pct': 125.0,      # 125% of secured target
        'secured_business_lakhs': 50.0,        # ₹50 lakh secured business

        'secured_direct_achievement_pct': 70.0, # 70% of direct target
        'secured_direct_business_lakhs': 20.0,  # ₹20 lakh direct business

        'unsecured_achievement_pct': 90.0,     # 90% of unsecured target
        'unsecured_business_lakhs': 30.0,      # ₹30 lakh unsecured business
    }

    return sample_bm

def demonstrate_step_by_step_calculation():
    """Demonstrate step-by-step calculation as per your example"""

    print("\n" + "="*80)
    print("STEP-BY-STEP CALCULATION DEMONSTRATION")
    print("="*80)

    # Your example: 14 months tenure, 5 secured disbursals, 1 unsecured disbursal
    tenure_months = 14
    secured_disbursals = 5
    unsecured_disbursals = 1

    calculator = NBFCIncentiveCalculator()

    print(f"Given Data:")
    print(f"  Tenure: {tenure_months} months")
    print(f"  Secured Disbursals: {secured_disbursals}")
    print(f"  Unsecured Disbursals: {unsecured_disbursals}")
    print()

    # Step 1: Find tenure slab for secured disbursals
    print("STEP 1: Finding Secured Effort Payout")
    print("-"*50)
    print("Tenure slabs available: (0,0), (1,6), (7,9), (10,∞)")
    print(f"Tenure {tenure_months} months falls in: (10,∞)")

    secured_grid = calculator.secured_fresh_disbursals[(10, float('inf'))]
    print(f"Disbursal grid for (10,∞): {secured_grid}")

    print(f"Looking for {secured_disbursals} disbursals in ranges:")
    for (start, end), payout in secured_grid.items():
        if start <= secured_disbursals <= end:
            print(f"  ✓ {secured_disbursals} falls in ({start},{end}): ₹{payout}")
            secured_payout = payout
        else:
            print(f"    {secured_disbursals} not in ({start},{end}): ₹{payout}")

    print()

    # Step 2: Find tenure slab for unsecured disbursals
    print("STEP 2: Finding Unsecured Effort Payout")
    print("-"*50)
    print("Tenure slabs available: (0,0), (1,14), (15,19), (20,23), (24,∞)")
    print(f"Tenure {tenure_months} months falls in: (1,14)")

    unsecured_grid = calculator.unsecured_fresh_disbursals[(1, 14)]
    print(f"Disbursal grid for (1,14): {unsecured_grid}")

    print(f"Looking for {unsecured_disbursals} disbursals in ranges:")
    for (start, end), payout in unsecured_grid.items():
        if start <= unsecured_disbursals <= end:
            print(f"  ✓ {unsecured_disbursals} falls in ({start},{end}): ₹{payout}")
            unsecured_payout = payout
        else:
            print(f"    {unsecured_disbursals} not in ({start},{end}): ₹{payout}")

    print()

    # Step 3: Final calculation
    print("STEP 3: Final Effort Payout Calculation")
    print("-"*50)
    print(f"Secured Effort Payout:   ₹{secured_payout:,}")
    print(f"Unsecured Effort Payout: ₹{unsecured_payout:,}")
    print(f"Total Effort Payout:     ₹{secured_payout + unsecured_payout:,}")
    print()

    return secured_payout, unsecured_payout

# Initialize calculator and run example
if __name__ == "__main__":

    # First demonstrate the step-by-step calculation as per your example
    demonstrate_step_by_step_calculation()

    # Then run the complete calculation
    calculator = NBFCIncentiveCalculator()

    # Create sample data
    sample_data = create_sample_data()

    print("COMPLETE SAMPLE CALCULATION:")
    print("-"*50)
    for key, value in sample_data.items():
        print(f"{key}: {value}")
    print()

    # Calculate incentive
    result = calculator.calculate_monthly_incentive(sample_data)

    # Print detailed breakdown
    calculator.print_detailed_breakdown(result)

    # Example of quarterly docking calculation
    print("\nQUARTERLY DOCKING EXAMPLE:")
    print("-"*50)
    quarterly_accumulated = 5000  # ₹5000 accumulated over quarter
    dpd_25_plus = 2.5  # 2.5% portfolio in DPD 25+

    docking_result = calculator.calculate_docking_release(quarterly_accumulated, dpd_25_plus)

    print(f"Accumulated Amount: ₹{docking_result['accumulated_amount']:,}")
    print(f"DPD 25+ Percentage: {docking_result['dpd_25_plus_percentage']}%")
    print(f"Release Factor: {docking_result['release_factor']*100}%")
    print(f"Released Amount: ₹{docking_result['released_amount']:,}")
    print(f"Forfeited Amount: ₹{docking_result['forfeited_amount']:,}")

    # Additional example with different data to show slab calculations
    print("\n" + "="*80)
    print("VOLUME PAYOUT SLAB DEMONSTRATION")
    print("="*80)

    # Example: 120% secured achievement with ₹100L business
    secured_ach = 120.0
    secured_business = 100.0

    print(f"Example: {secured_ach}% achievement with ₹{secured_business}L business")
    print("Secured Business Slabs:")
    for start, end, rate in calculator.secured_business_slabs:
        print(f"  {start}-{end}%: ₹{rate} per lakh")

    print("\nCalculation:")
    print(f"  0-60%: No payout (below threshold)")
    print(f"  60-80%: 20% × ₹{secured_business}L × ₹450 = ₹{20 * secured_business * 450 / 100:,.0f}")
    print(f"  80-100%: 20% × ₹{secured_business}L × ₹500 = ₹{20 * secured_business * 500 / 100:,.0f}")
    print(f"  100-120%: 20% × ₹{secured_business}L × ₹600 = ₹{20 * secured_business * 600 / 100:,.0f}")

    total_volume = (20 * secured_business * 450 / 100) + (20 * secured_business * 500 / 100) + (20 * secured_business * 600 / 100)
    print(f"  Total Volume Payout: ₹{total_volume:,.0f}")
    print(f"  Immediate Release (85%): ₹{total_volume * 0.85:,.0f}")
    print(f"  Quarterly Accumulation (15%): ₹{total_volume * 0.15:,.0f}")

## Try 4

In [22]:
import math

class BMIncentiveCalculator:

    def __init__(self):
        self.secured_fresh_disbursals = {
            (0, 0): {(0, 0): 0, (1, 2): 1400, (3, 4): 1800, (5, 6): 2200, (7, float('inf')): 2400},
            (1, 6): {(0, 0): 1200, (1, 2): 1600, (3, 4): 2000, (5, 6): 2500, (7, float('inf')): 2800},
            (7, 9): {(0, 0): 1400, (1, 2): 1800, (3, 4): 2500, (5, 6): 3000, (7, float('inf')): 4000},
            (10, float('inf')): {(0, 0): 1800, (1, 2): 2200, (3, 4): 3000, (5, 6): 3500, (7, float('inf')): 5000}
        }

        self.unsecured_fresh_disbursals = {
            (0, 0): {(0, 0): 0, (1, 3): 250, (4, 7): 350, (8, 9): 450, (10, 13): 550, (14, float('inf')): 700},
            (1, 14): {(0, 0): 200, (1, 3): 300, (4, 7): 400, (8, 9): 500, (10, 13): 600, (14, float('inf')): 750},
            (15, 19): {(0, 0): 250, (1, 3): 350, (4, 7): 450, (8, 9): 550, (10, 13): 650, (14, float('inf')): 850},
            (20, 23): {(0, 0): 300, (1, 3): 400, (4, 7): 500, (8, 9): 600, (10, 13): 700, (14, float('inf')): 1000},
            (24, float('inf')): {(0, 0): 500, (1, 3): 600, (4, 7): 700, (8, 9): 1000, (10, 13): 1200, (14, float('inf')): 1500}
        }

        self.secured_business_slabs = [
            (0, 60, 0),
            (60, 80, 450),
            (80, 100, 500),
            (100, 150, 600),
            (150, 1000, 750)
        ]

        self.secured_direct_business_slabs = [
            (0, 40, 50),
            (40, 60, 100),
            (60, 80, 150),
            (80, 1000, 200)
        ]

        self.unsecured_business_slabs = [
            (0, 60, 0),
            (60, 80, 300),
            (80, 100, 400),
            (100, 150, 500),
            (150, 1000, 600)
        ]

    def _get_bonus_from_matrix(self, matrix, logins, disbursals):
        for login_range, disbursal_bonuses in matrix.items():
            login_min, login_max = login_range
            if login_min <= logins <= login_max:
                for disbursal_range, bonus_amount in disbursal_bonuses.items():
                    disbursal_min, disbursal_max = disbursal_range
                    if disbursal_min <= disbursals <= disbursal_max:
                        return bonus_amount
        return 0

    def _calculate_volume_bonus_tax_slab(self, total_disbursal_amount, target_amount, slabs):
        if target_amount <= 0:
            return 0

        volume_bonus = 0

        sorted_slabs = sorted(slabs, key=lambda x: x[0])

        for lower_perc, upper_perc, rate in sorted_slabs:

            lower_bound_amount = (lower_perc / 100.0) * target_amount
            upper_bound_amount = (upper_perc / 100.0) * target_amount

            amount_in_current_slab_band = max(0, min(total_disbursal_amount, upper_bound_amount) - lower_bound_amount)
            print("Lower : ", lower_bound_amount," Upper : ",upper_bound_amount," Left : ",amount_in_current_slab_band)
            if total_disbursal_amount <= lower_bound_amount:
                continue

            if rate > 0:
                volume_bonus += (amount_in_current_slab_band / 100000.0) * rate
        print("\n")
        return volume_bonus

    def _calculate_volume_bonus_simple_rate(self, total_disbursal_amount, achievement_percentage, slabs):
        for lower_perc, upper_perc, rate in slabs:
            if lower_perc <= achievement_percentage <= upper_perc:
                return (total_disbursal_amount / 100000.0) * rate
        return 0

    def calculate_bonus(self, secured_logins, unsecured_logins, secured_disbursals,
                        unsecured_disbursals, total_secured_disbursal_amount,
                        secured_direct_percentage, secured_target, unsecured_target,
                        total_unsecured_disbursal_amount=0):
        # --- Effort Bonus Calculation ---
        secured_effort_bonus = self._get_bonus_from_matrix(
            self.secured_fresh_disbursals, secured_logins, secured_disbursals
        )

        unsecured_effort_bonus = self._get_bonus_from_matrix(
            self.unsecured_fresh_disbursals, unsecured_logins, unsecured_disbursals
        )

        total_effort_bonus = secured_effort_bonus + unsecured_effort_bonus

        # --- Volume Bonus Calculation ---

        # Secured Business Volume Bonus (using tax slab logic)
        secured_volume_bonus = self._calculate_volume_bonus_tax_slab(
            total_secured_disbursal_amount, secured_target, self.secured_business_slabs
        )

        # Secured Direct Business Volume Bonus (using simple rate logic)
        secured_direct_volume_bonus = self._calculate_volume_bonus_simple_rate(
            total_secured_disbursal_amount, secured_direct_percentage, self.secured_direct_business_slabs
        )

        # Unsecured Business Volume Bonus (using tax slab logic, if applicable)
        unsecured_volume_bonus = self._calculate_volume_bonus_tax_slab(
            total_unsecured_disbursal_amount, unsecured_target, self.unsecured_business_slabs
        )

        total_volume_bonus = secured_volume_bonus + secured_direct_volume_bonus + unsecured_volume_bonus

        total_bonus = total_effort_bonus + total_volume_bonus

        return {
            "secured_effort_bonus": secured_effort_bonus,
            "unsecured_effort_bonus": unsecured_effort_bonus,
            "total_effort_bonus": total_effort_bonus,
            "secured_volume_bonus": secured_volume_bonus,
            "secured_direct_volume_bonus": secured_direct_volume_bonus,
            "unsecured_volume_bonus": unsecured_volume_bonus,
            "total_volume_bonus": total_volume_bonus,
            "overall_total_bonus": total_bonus
        }

In [23]:
# --- Example Usage ---
# Instantiate the calculator
calculator = BMIncentiveCalculator()

# Input data from your example
secured_logins = 14
unsecured_logins = 1
secured_disbursals = 5
unsecured_disbursals = 0
total_secured_disbursal_amount = 3906813  # Rs. 39,06,813
secured_direct_percentage = 100          # 100% secured direct
secured_target = 4275000                 # Rs. 42,75,000
unsecured_target = 225000                # Rs. 2,25,000
total_unsecured_disbursal_amount_example = 0 # Not provided in example, assuming 0 for now

# Calculate the bonus
bonus_results = calculator.calculate_bonus(
    secured_logins=secured_logins,
    unsecured_logins=unsecured_logins,
    secured_disbursals=secured_disbursals,
    unsecured_disbursals=unsecured_disbursals,
    total_secured_disbursal_amount=total_secured_disbursal_amount,
    secured_direct_percentage=secured_direct_percentage,
    secured_target=secured_target,
    unsecured_target=unsecured_target,
    total_unsecured_disbursal_amount=total_unsecured_disbursal_amount_example
)

# Print the results
print("--- Bonus Calculation Results ---")
print(f"Secured Effort Bonus: {bonus_results['secured_effort_bonus']:.2f}")
print(f"Unsecured Effort Bonus: {bonus_results['unsecured_effort_bonus']:.2f}")
print(f"Total Effort Bonus: {bonus_results['total_effort_bonus']:.2f}")
print("-" * 30)
print(f"Secured Volume Bonus (General): {bonus_results['secured_volume_bonus']:.2f}")
print(f"Secured Direct Volume Bonus: {bonus_results['secured_direct_volume_bonus']:.2f}")
print(f"Unsecured Volume Bonus: {bonus_results['unsecured_volume_bonus']:.2f}")
print(f"Total Volume Bonus: {bonus_results['total_volume_bonus']:.2f}")
print("-" * 30)
print(f"Overall Total Bonus: {bonus_results['overall_total_bonus']:.2f}")

Lower :  0.0  Upper :  2565000.0  Left :  2565000.0
Lower :  2565000.0  Upper :  3420000.0  Left :  855000.0
Lower :  3420000.0  Upper :  4275000.0  Left :  486813.0
Lower :  4275000.0  Upper :  6412500.0  Left :  0
Lower :  6412500.0  Upper :  42750000.0  Left :  0


Lower :  0.0  Upper :  135000.0  Left :  0
Lower :  135000.0  Upper :  180000.0  Left :  0
Lower :  180000.0  Upper :  225000.0  Left :  0
Lower :  225000.0  Upper :  337500.0  Left :  0
Lower :  337500.0  Upper :  2250000.0  Left :  0


--- Bonus Calculation Results ---
Secured Effort Bonus: 3500.00
Unsecured Effort Bonus: 200.00
Total Effort Bonus: 3700.00
------------------------------
Secured Volume Bonus (General): 6281.57
Secured Direct Volume Bonus: 7813.63
Unsecured Volume Bonus: 0.00
Total Volume Bonus: 14095.19
------------------------------
Overall Total Bonus: 17795.19

--- Example Breakdown from your request ---
Expected Effort Bonus: 3500(sec) + 200(unsec) = 3700
My Calculated Effort Bonus: 3500.00 (sec) + 2

## Test 5

In [30]:
import math

class BMIncentiveCalculator:

    def __init__(self):
        self.secured_fresh_disbursals = {
            (0, 0): {(0, 0): 0, (1, 2): 1400, (3, 4): 1800, (5, 6): 2200, (7, float('inf')): 2400},
            (1, 6): {(0, 0): 1200, (1, 2): 1600, (3, 4): 2000, (5, 6): 2500, (7, float('inf')): 2800},
            (7, 9): {(0, 0): 1400, (1, 2): 1800, (3, 4): 2500, (5, 6): 3000, (7, float('inf')): 4000},
            (10, float('inf')): {(0, 0): 1800, (1, 2): 2200, (3, 4): 3000, (5, 6): 3500, (7, float('inf')): 5000}
        }

        self.unsecured_fresh_disbursals = {
            (0, 0): {(0, 0): 0, (1, 3): 250, (4, 7): 350, (8, 9): 450, (10, 13): 550, (14, float('inf')): 700},
            (1, 14): {(0, 0): 200, (1, 3): 300, (4, 7): 400, (8, 9): 500, (10, 13): 600, (14, float('inf')): 750},
            (15, 19): {(0, 0): 250, (1, 3): 350, (4, 7): 450, (8, 9): 550, (10, 13): 650, (14, float('inf')): 850},
            (20, 23): {(0, 0): 300, (1, 3): 400, (4, 7): 500, (8, 9): 600, (10, 13): 700, (14, float('inf')): 1000},
            (24, float('inf')): {(0, 0): 500, (1, 3): 600, (4, 7): 700, (8, 9): 1000, (10, 13): 1200, (14, float('inf')): 1500}
        }

        self.secured_business_slabs = [
            (0, 60, 0),
            (60, 80, 450),
            (80, 100, 500),
            (100, 150, 600),
            (150, 1000, 750)
        ]

        self.secured_direct_business_slabs = [
            (0, 40, 50),
            (40, 60, 100),
            (60, 80, 150),
            (80, 1000, 200)
        ]

        self.unsecured_business_slabs = [
            (0, 60, 0),
            (60, 80, 300),
            (80, 100, 400),
            (100, 150, 500),
            (150, 1000, 600)
        ]

    def _get_bonus_from_matrix(self, matrix, logins, disbursals):
        # print(f"\n--- Effort Bonus Calculation (Logins: {logins}, Disbursals: {disbursals}) ---")
        for login_range, disbursal_bonuses in matrix.items():
            login_min, login_max = login_range
            # print(f"Checking login range: ({login_min}, {login_max})")
            if login_min <= logins <= login_max:
                # print(f"  Login {logins} falls in range ({login_min}, {login_max}).")
                for disbursal_range, bonus_amount in disbursal_bonuses.items():
                    disbursal_min, disbursal_max = disbursal_range
                    # print(f"    Checking disbursal range: ({disbursal_min}, {disbursal_max})")
                    if disbursal_min <= disbursals <= disbursal_max:
                        # print(f"      Disbursal {disbursals} falls in range ({disbursal_min}, {disbursal_max}).")
                        # print(f"      Matched bonus amount: {bonus_amount}")
                        return bonus_amount
        print("  No matching bonus found for given logins and disbursals.")
        return 0

    def _calculate_volume_bonus_tax_slab(self, total_disbursal_amount, target_amount, slabs):
        # print(f"\n--- Volume Bonus Calculation (Amount: {total_disbursal_amount}, Target: {target_amount}) ---")
        if target_amount <= 0:
            print("Target amount is zero or less, volume bonus is 0.")
            return 0

        achievement_percentage = (total_disbursal_amount / target_amount) * 100
        # print(f"Achievement Percentage: {achievement_percentage:.2f}%")

        effective_rate = 0
        first_non_zero_slab_lower_bound_amount = 0
        found_first_non_zero = False

        # Sort slabs by lower bound of percentage for correct sequential processing
        sorted_slabs = sorted(slabs, key=lambda x: x[0])

        for lower_perc, upper_perc, rate in sorted_slabs:
            # Calculate the actual amount thresholds for this slab based on the target
            lower_bound_amount = (lower_perc / 100.0) * target_amount
            upper_bound_amount = (upper_perc / 100.0) * target_amount

            # print(f"  Slab: ({lower_perc:.0f}%, {upper_perc:.0f}%) | Rate: {rate} | Lower Amt: {lower_bound_amount:.2f} | Upper Amt: {upper_bound_amount:.2f}")

            # Determine the effective rate based on the highest slab achieved
            if lower_perc <= achievement_percentage <= upper_perc:
                effective_rate = rate
                # print(f"    Achievement {achievement_percentage:.2f}% falls in this slab. Effective rate set to {effective_rate}.")

            # Find the lower bound of the first slab with a non-zero rate
            if rate > 0 and not found_first_non_zero:
                first_non_zero_slab_lower_bound_amount = lower_bound_amount
                found_first_non_zero = True
                # print(f"    First non-zero rate slab starts at {first_non_zero_slab_lower_bound_amount:.2f}.")

        volume_bonus = 0
        if effective_rate > 0:
            # Calculate bonus on the amount above the lower bound of the first non-zero rate slab
            # Ensure we don't calculate bonus on negative amounts if disbursed amount is less than threshold
            amount_for_bonus = max(0, total_disbursal_amount - first_non_zero_slab_lower_bound_amount)
            volume_bonus = (amount_for_bonus / 100000.0) * effective_rate
            # print(f"  Amount eligible for bonus (above first non-zero slab threshold): {amount_for_bonus:.2f}")
            # print(f"  Calculated Volume Bonus: ({amount_for_bonus:.2f} / 100000.0) * {effective_rate} = {volume_bonus:.2f}")
        else:
            print("  No effective rate or achievement too low for bonus.")

        return volume_bonus

    def _calculate_volume_bonus_simple_rate(self, total_disbursal_amount, achievement_percentage, slabs):
        # print(f"\n--- Simple Rate Volume Bonus Calculation (Amount: {total_disbursal_amount}, Percentage: {achievement_percentage:.2f}%) ---")
        for lower_perc, upper_perc, rate in slabs:
            # print(f"  Checking slab: ({lower_perc}%, {upper_perc}%) | Rate: {rate}")
            if lower_perc <= achievement_percentage <= upper_perc:
                bonus = (total_disbursal_amount / 100000.0) * rate
                # print(f"  Achievement {achievement_percentage:.2f}% falls in range ({lower_perc}%, {upper_perc}%).")
                # print(f"  Calculated Bonus: ({total_disbursal_amount:.2f} / 100000.0) * {rate} = {bonus:.2f}")
                return bonus
        print("  No matching bonus rate found for the achievement percentage.")
        return 0

    def calculate_bonus(self, secured_logins, unsecured_logins, secured_disbursals,
                        unsecured_disbursals, total_secured_disbursal_amount,
                        secured_direct_percentage, secured_target, unsecured_target,
                        total_unsecured_disbursal_amount=0):
        # --- Effort Bonus Calculation ---
        secured_effort_bonus = self._get_bonus_from_matrix(
            self.secured_fresh_disbursals, secured_logins, secured_disbursals
        )

        unsecured_effort_bonus = self._get_bonus_from_matrix(
            self.unsecured_fresh_disbursals, unsecured_logins, unsecured_disbursals
        )

        total_effort_bonus = secured_effort_bonus + unsecured_effort_bonus

        # --- Volume Bonus Calculation ---

        # Secured Business Volume Bonus (using stepped-up bonus logic)
        secured_volume_bonus = self._calculate_volume_bonus_tax_slab(
            total_secured_disbursal_amount, secured_target, self.secured_business_slabs
        )

        # Secured Direct Business Volume Bonus (using simple rate logic)
        secured_direct_volume_bonus = self._calculate_volume_bonus_simple_rate(
            total_secured_disbursal_amount, secured_direct_percentage, self.secured_direct_business_slabs
        )

        # Unsecured Business Volume Bonus (using stepped-up bonus logic, if applicable)
        unsecured_volume_bonus = self._calculate_volume_bonus_tax_slab(
            total_unsecured_disbursal_amount, unsecured_target, self.unsecured_business_slabs
        )

        total_volume_bonus = secured_volume_bonus + secured_direct_volume_bonus + unsecured_volume_bonus

        total_bonus = total_effort_bonus + total_volume_bonus

        return {
            "secured_effort_bonus": secured_effort_bonus,
            "unsecured_effort_bonus": unsecured_effort_bonus,
            "total_effort_bonus": total_effort_bonus,
            "secured_volume_bonus": secured_volume_bonus,
            "secured_direct_volume_bonus": secured_direct_volume_bonus,
            "unsecured_volume_bonus": unsecured_volume_bonus,
            "total_volume_bonus": total_volume_bonus,
            "overall_total_bonus": total_bonus
        }

In [32]:
# --- Example Usage ---
calculator = BMIncentiveCalculator()

secured_logins = 20
unsecured_logins = 0
secured_disbursals = 1
unsecured_disbursals = 0
total_secured_disbursal_amount = 2000000
secured_direct_percentage = 100
secured_target =  3325000
unsecured_target =  175000
total_unsecured_disbursal_amount_example = 0

bonus_results = calculator.calculate_bonus(
    secured_logins=secured_logins,
    unsecured_logins=unsecured_logins,
    secured_disbursals=secured_disbursals,
    unsecured_disbursals=unsecured_disbursals,
    total_secured_disbursal_amount=total_secured_disbursal_amount,
    secured_direct_percentage=secured_direct_percentage,
    secured_target=secured_target,
    unsecured_target=unsecured_target,
    total_unsecured_disbursal_amount=total_unsecured_disbursal_amount_example
)

# Print the results
print("\n--- Final Bonus Calculation Results ---")
print(f"Secured Effort Bonus: {bonus_results['secured_effort_bonus']:.2f}")
print(f"Unsecured Effort Bonus: {bonus_results['unsecured_effort_bonus']:.2f}")
print(f"Total Effort Bonus: {bonus_results['total_effort_bonus']:.2f}")
print("-" * 30)
print(f"Secured Volume Bonus (General): {bonus_results['secured_volume_bonus']:.2f}")
print(f"Secured Direct Volume Bonus: {bonus_results['secured_direct_volume_bonus']:.2f}")
print(f"Unsecured Volume Bonus: {bonus_results['unsecured_volume_bonus']:.2f}")
print(f"Total Volume Bonus: {bonus_results['total_volume_bonus']:.2f}")
print("-" * 30)
print(f"Overall Total Bonus: {bonus_results['overall_total_bonus']:.2f}")

  No effective rate or achievement too low for bonus.

--- Final Bonus Calculation Results ---
Secured Effort Bonus: 2200.00
Unsecured Effort Bonus: 0.00
Total Effort Bonus: 2200.00
------------------------------
Secured Volume Bonus (General): 22.50
Secured Direct Volume Bonus: 4000.00
Unsecured Volume Bonus: 0.00
Total Volume Bonus: 4022.50
------------------------------
Overall Total Bonus: 6222.50
